In [6]:
import heapq
import urllib
import os
from collections import defaultdict
from bitarray import bitarray
import cPickle as pickle


# Download the file if need be:
def download_file(url, filename):
    if not os.path.exists(filename):
        urllib.urlretrieve(url + filename, filename)


# build a frequency table:
def build_freq(filename):
    freq = defaultdict(int)
    with open(filename, 'r') as f:
        for line in f:
            for char in line('utf-8-sig'):
                freq[char] += 1
    total = float(sum(freq.values()))
    return {char: count / total for (char, count) in freq.items()}


# Now build the Huffman encoding:
def encode(symb2freq):
    # return pair (sym2object, tree), where
    # sym2object is a dict mapping a symbol name to its Symbol object,
    # and tree is the root of the Huffman tree


    raise NotImplementedError()
    heap = [[wt, [sym, bitarray()]] for sym, wt in symb2freq.items()]
    (true, false) = (bitarray('1'), bitarray('0'))
    heapq.heapify(heap)
    while len(heap) > 1:
        lo = heapq.heappop(heap)
        hi = heapq.heappop(heap)
        for pair in lo[1:]:
            pair[1] = false + pair[1]
        for pair in hi[1:]:
            pair[1] = true + pair[1]
        heapq.heappush(heap, [lo[0] + hi[0]] + lo[1:] + hi[1:])
    return dict(heapq.heappop(heap)[1:])


# Now compress the file:
def compress(filename, encoding, compressed_name=None):
    if compressed_name is None:
        compressed_name = filename + ".huff"
    output = bitarray()
    with open(filename, 'r') as f:
        for line in f:
            for char in line:
                output.extend(encoding[char])
    N = len(output)
    with open(compressed_name, 'wb') as f:
        pickle.dump(N, f)
        pickle.dump(encoding, f)
        output.tofile(f)


# Now decompress the file:
def decompress(filename, decompressed_name=None):
    if decompressed_name is None:
        decompressed_name = filename + ".dehuff"
    with open(filename, 'rb') as f:
        N = pickle.load(f)
        encoding = pickle.load(f)
        bits = bitarray()
        bits.fromfile(f)
        bits = bits[:N]

    # Totally cheating here and using a builtin method:
    #output = bits.decode(encoding)

    output = "".join(output).encode('utf-8-sig')
    with open(decompressed_name, 'wb') as f:
        f.write(output)


# url = "https://www.gutenberg.org/ebooks/"
# filename = "100.txt.utf-8"

url = "http://www.gutenberg.org/files/100/"
filename = "100-0.txt"

download_file(url, filename)
freq = build_freq(filename)
encoding = encode(freq)
compress(filename, encoding)
decompress(filename + ".huff")
# Do you get identical files?

TypeError: 'str' object is not callable

In [ ]:
16.3-2 Prove that a binary tree that is not full cannot correspond to an optimal prefix code.

This implies that there exists at least one node (b) with only one child ©. If b is the root with child c, b can be deleted and c used as the root to the same effect. If b is not the root, it can still be deleted, c becomes the new leaf. In both scenarios, the length from root to leaf decreased, and the number of bits to store became smaller. If this is always possible for an incomplete tree, then an incomplete tree is never optimal.

16.3-3 What is an optimal Huffman code for the following set of frequencies, based on the first 8 Fibonacci numbers? {a:1 b:1 c:2 d:3 e:5 f:8 g:13 h:21} Can you generalize your answer to find the optimal code when the frequencies are the first n Fibonacci numbers? a: 1111111; b: 1111110; c: 111110; d: 11110; e: 1110; f: 110; g: 10; h: 0; a: 1*(n-1); b: 1*(n-2) + 0; c: 1*(n-3) + 0